In [240]:
!pip install scikeras

     |████████████████████████████████| 8.6 MB 5.4 MB/s eta 0:00:01     |███████████████████████▊        | 6.4 MB 5.4 MB/s eta 0:00:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.24.1
    Uninstalling scikit-learn-0.24.1:
      Successfully uninstalled scikit-learn-0.24.1


In [241]:
#import packages

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [2]:
#read in data

df=pd.read_csv('TikTok_songs_2022.csv')

In [3]:
#preview data

df.head()

,track_name,artist_name,artist_pop,album,track_pop,danceability,energy,loudness,mode,key,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,duration_ms
0,Running Up That Hill (A Deal With God),Kate Bush,81,Hounds Of Love,95,0.629,0.547,-13.123,0,10,0.0550,0.7200,0.003140,0.0604,0.197,108.375,4,298933
1,As It Was,Harry Styles,91,As It Was,96,0.520,0.731,-5.338,0,6,0.0557,0.3420,0.001010,0.3110,0.662,173.930,4,167303
2,Sunroof,Nicky Youre,73,Sunroof,44,0.768,0.716,-5.110,1,10,0.0404,0.3500,0.000000,0.1500,0.841,131.430,4,163026
3,Heat Waves,Glass Animals,80,Dreamland (+ Bonus Levels),89,0.761,0.525,-6.900,1,11,0.0944,0.4400,0.000007,0.0921,0.531,80.870,4,238805
4,About Damn Time,Lizzo,81,About Damn Time,92,0.836,0.743,-6.305,0,10,0.0656,0.0995,0.000000,0.3350,0.722,108.966,4,191822


# Preprocessing 

In [5]:
#summary statistics

df.describe()

,artist_pop,track_pop,danceability,energy,loudness,mode,key,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,duration_ms
count,263.000000,263.000000,263.000000,263.000000,263.000000,263.000000,263.000000,263.000000,263.000000,263.000000,263.000000,263.000000,263.000000,263.000000,263.000000
mean,64.186312,59.844106,0.715338,0.682631,-6.300513,0.536122,5.809886,0.117268,0.193188,0.020244,0.202562,0.506149,122.883696,3.969582,174656.212928
std,18.146338,24.976524,0.117722,0.141756,1.905114,0.499644,3.657312,0.101417,0.214588,0.109370,0.151808,0.220757,24.396686,0.288168,34785.125564
min,18.000000,0.000000,0.308000,0.214000,-13.468000,0.000000,0.000000,0.025200,0.000038,0.000000,0.026500,0.036300,62.948000,1.000000,85742.000000
25%,50.000000,47.000000,0.646000,0.593000,-7.423500,0.000000,2.000000,0.044750,0.028100,0.000000,0.094350,0.349000,106.850500,4.000000,148646.000000
50%,64.000000,66.000000,0.726000,0.701000,-6.067000,1.000000,6.000000,0.070700,0.120000,0.000003,0.132000,0.485000,123.056000,4.000000,171028.000000
75%,81.000000,79.500000,0.796500,0.790500,-4.887000,1.000000,9.000000,0.156000,0.298500,0.000334,0.277500,0.680000,130.004500,4.000000,199047.000000
max,95.000000,97.000000,0.961000,0.955000,-2.634000,1.000000,11.000000,0.481000,0.965000,0.939000,0.790000,0.956000,187.906000,5.000000,298933.000000


In [98]:
#Labeling data into X and Y

X=df.drop('track_pop', axis=1)
Y=df['track_pop']

In [268]:
#splitting data into training and testing 
#doing a small test set due to small amount of data

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=.15, random_state=41)

In [269]:
#get only numerical columns for modeling

x_train2=x_train[x_train.columns.difference(['track_name', 'artist_name','album'])]
x_test2=x_test[x_test.columns.difference(['track_name', 'artist_name','album'])]

# Feature Selection

In [270]:
#recursive feature elimination to further narrow down variables 

# create pipeline
rfe = RFECV(estimator=DecisionTreeClassifier())
model = DecisionTreeClassifier()
pipeline = Pipeline(steps=[('s',rfe),('m',model)])

In [271]:
# fit RFE

rfe.fit(x_train2, y_train)
# summarize all features
for i in range(x_train2.shape[1]):
    print('Column: %s, Selected %s, Rank: %.3f' % (list(x_train2.columns)[i], rfe.support_[i], rfe.ranking_[i]))

/Users/sarahrussell/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


Column: acousticness, Selected True, Rank: 1.000
Column: artist_pop, Selected True, Rank: 1.000
Column: danceability, Selected True, Rank: 1.000
Column: duration_ms, Selected False, Rank: 2.000
Column: energy, Selected True, Rank: 1.000
Column: instrumentalness, Selected False, Rank: 5.000
Column: key, Selected False, Rank: 3.000
Column: liveness, Selected True, Rank: 1.000
Column: loudness, Selected True, Rank: 1.000
Column: mode, Selected False, Rank: 4.000
Column: speechiness, Selected True, Rank: 1.000
Column: tempo, Selected True, Rank: 1.000
Column: time_signature, Selected False, Rank: 6.000
Column: valence, Selected True, Rank: 1.000


In [272]:
# keep variables where selected =true

var_list=[]

for i in range(x_train2.shape[1]):
    if rfe.support_[i]==True:
        var_list.append(list(x_train2.columns)[i])
    else: continue
    
x_train_2=x_train2[var_list]
x_test_2=x_test2[var_list]

In [273]:
#preview final training set and final variables
x_train_2.head()

,acousticness,artist_pop,danceability,energy,liveness,loudness,speechiness,tempo,valence
260,0.000307,95,0.529,0.673,0.093,-4.711,0.1750,165.921,0.366
206,0.435000,58,0.826,0.615,0.334,-4.729,0.1370,102.999,0.841
46,0.329000,41,0.776,0.656,0.119,-8.921,0.1300,148.950,0.649
57,0.032100,64,0.579,0.727,0.714,-5.149,0.0399,124.090,0.246
117,0.090500,35,0.552,0.831,0.440,-7.228,0.1940,127.851,0.733


In [274]:
#scale data for modeling

# define standard scaler
scaler = StandardScaler()

#scale data
x_train2_scaled= scaler.fit_transform(x_train_2)
x_test2_scaled= scaler.fit_transform(x_test_2)
y_train_scaled= scaler.fit_transform(np.array(y_train).reshape(-1,1))
y_test_scaled= scaler.fit_transform(np.array(y_test).reshape(-1,1))

# Linear Regression

In [279]:
lr = linear_model.LinearRegression()
lr.fit(x_train2_scaled, y_train_scaled)

LinearRegression()

In [280]:
r_sq = lr.score(x_train2_scaled, y_train_scaled)
lr.score(x_train2_scaled, y_train_scaled)

0.2722554414308368

In [281]:
#accuracy on test

lr.score(x_test2_scaled, y_test_scaled)

0.4147521810158823

In [282]:
#not great lets move on

# Random Forest

In [301]:
rf = RandomForestRegressor(max_depth=3, random_state=0,
                          n_estimators=100)
rf.fit(x_train2_scaled, y_train_scaled)

<ipython-input-301-64241206b6f9>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(x_train2_scaled, y_train_scaled)


RandomForestRegressor(max_depth=3, random_state=0)

In [302]:
rf.score(x_train2_scaled, y_train_scaled)

0.516607198221927

In [303]:
rf.score(x_test2_scaled, y_test_scaled)

0.3959378728315366

In [304]:
#still not great after hyperparameter tuning, let's move on

# Gradient Boosting

In [318]:
gb = GradientBoostingRegressor(random_state=0, subsample=.6,
                              n_estimators=50, learning_rate=.1)
gb.fit(x_train2_scaled, y_train_scaled)

/Users/sarahrussell/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


GradientBoostingRegressor(n_estimators=50, random_state=0, subsample=0.6)

In [319]:
gb.score(x_train2_scaled, y_train_scaled)

0.7744916996583533

In [320]:
gb.score(x_test2_scaled, y_test_scaled)

0.3036691102365856

In [321]:
#overfitting after adjusting hyperparamets so moving on

# Lasso Regression

In [322]:
clf = linear_model.Lasso(alpha=0.1)
clf.fit(x_train2_scaled, y_train_scaled)

Lasso(alpha=0.1)

In [323]:
clf.score(x_train2_scaled, y_train_scaled)

0.24422525405702833

In [324]:
clf.score(x_test2_scaled, y_test_scaled)

0.37999068751212595

# Neural Network

In [325]:
x_train2_scaled.shape

(223, 9)

In [329]:
# define base model
def nn_model():
    seq = Sequential()
    seq.add(Dense(9, input_shape=(9,), kernel_initializer='normal', activation='relu'))
    seq.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    seq.compile(loss='mean_squared_error', optimizer='adam')
    return seq

In [330]:
# evaluate model
estimator = KerasRegressor(model=nn_model, epochs=100, batch_size=5, verbose=0)
kfold = KFold(n_splits=10)

results = cross_val_score(estimator, x_train2_scaled, y_train_scaled, cv=kfold, scoring='neg_mean_squared_error')

In [331]:
results.mean()

-0.9127675971836979

In [332]:
estimator.fit(x_train2_scaled, y_train_scaled)

KerasRegressor(
	model=<function nn_model at 0x7f8788383af0>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=5
	validation_batch_size=None
	verbose=0
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=100
)

In [ ]:
# will use keras model for predictions as other models did not do great

In [348]:
final=pd.DataFrame() 

In [349]:
predictions=list(estimator.predict(x_test2_scaled))

In [351]:
predictions = scaler.inverse_transform(predictions)

In [352]:
final['song']=list(x_test['track_name'])
final['predictions']=predictions
final['actual']=list(y_test)

In [354]:
final.sort_values('predictions', ascending=False)

,song,predictions,actual
17,drivers license,89.142448,29
16,MIDDLE OF THE NIGHT,84.502510,91
26,Heat Waves,82.302177,89
35,Levitating (feat. DaBaby),82.015411,85
33,WITHOUT YOU,81.984222,80
32,Savage,76.524460,70
23,ROCKSTAR (feat. Roddy Ricch),76.445641,80
3,Sweater Weather,75.486137,90
39,Todo De Ti,74.529205,82
4,Wait a Minute!,73.346153,86


# Next Steps

Ideally we would gather more data to train and test our model on. Additionally, more hyperparamters within each model could be looped through and tested to find the best fit for each model. It would also be good to look into 'Driver's License' to udnerstand why the Keras model got that so wrong. 